In [1]:
!pip install us

In [2]:
import us
states = us.STATES
states_clean = []
for state in states:
    states_clean.append(str(state).replace(" ", "_"))
states_clean.append('Washington_DC')


In [26]:
import urllib.request
import bs4
import pandas as pd
import numpy as np
import pickle
import re
from tqdm.notebook import tqdm

site_url = 'https://www.shakeshack.com/locations/'

states = []
names = []
addresses = []

ignore_locs = ['Valley Fair, CA']



user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers = {'User-Agent':user_agent,} 
request = urllib.request.Request(site_url,None,headers)
response = urllib.request.urlopen(request)
webpage = response.read()
soup = bs4.BeautifulSoup(webpage, 'html.parser')

for state in states_clean:
    state_tag = 'usa_' + state
    body = soup.find("div", id = state_tag)
    if body is not None:
        locs = body.find_all("div", class_="span5 item")
        for loc in locs:
            if loc.a.text not in ignore_locs:
                states.append(state)
                names.append(loc.a.text)
                addresses.append(loc.p.text)




df = pd.DataFrame.from_dict({'state':states, 'name':names, 'address':addresses})
df.replace({'_': ' ', '\n':', '}, regex=True, inplace=True)


In [31]:
df.to_excel('shakeshack.xlsx', index=False)

In [19]:
!pip install geopy
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = 'jeff')

def get_lat_long(address, l_type):
    loc = geolocator.geocode(address)
    if loc is None:
        return np.nan
    elif l_type == 0:
        return loc.latitude
    else:
        return loc.longitude

df[0:10]['address'].apply(lambda x: get_lat_long(x, 0))
df[0:10]['address'].apply(lambda x: get_lat_long(x, 1))

0           NaN
1   -111.931769
2   -111.925908
3    -70.425318
4           NaN
5           NaN
6   -118.309809
7           NaN
8           NaN
9   -118.256065
Name: address, dtype: float64

In [12]:
df.replace({'\n': ', '}, regex=True, inplace=True)

In [25]:
df

,state,name,address,latitude
0,Alabama,"Birmingham, AL","200 Summit Blvd. Suite 100, Birmingham, Alabam...",NaN
1,Arizona,"Fashion Square, AZ",7014 E. Camelback Road,33.503068
2,Arizona,"Kierland Commons, AZ",15030 N Scottsdale Road,33.589341
3,Arizona,"Phoenix Uptown Plaza, AZ",100 E. Camelback Road,41.683355
4,Arizona,Phoenix Sky Harbor International Airport,"Phoenix Sky Harbor International Airport, Term...",NaN
...,...,...,...,...
188,Washington_DC,"Logan Circle, Washington, D.C.",1400 14th Street NW,38.908981
189,Washington_DC,"Nationals Park, Washington, D.C.","1500 South Capitol Street, SE",39.932950
190,Washington_DC,"Navy Yard, Washington, D.C.",54 M Street SE,38.876564
191,Washington_DC,"The Wharf, Washington, D.C.","975 Wharf Street SW, Washington, D.C. 20024, 2...",NaN
